In [2]:
import nltk

In [3]:
import numpy as np
import pandas as pd
import string
string.punctuation
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [7]:
df = pd.read_csv('/content/IMDB Dataset.csv', encoding='latin-1')

In [8]:
df.sample(5)

,review,sentiment
32136,When all we have anymore is pretty much realit...,negative
30705,"This was, without a doubt, the worse horror mo...",negative
33307,"twenty years later, this movie still remains a...",positive
45417,Your idol will deceive you in this movie. Step...,negative
26603,I work at Memorial Hermann Hospital (TMC) and ...,negative


In [9]:
len(df)

50000

In [10]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [11]:
from nltk.stem import WordNetLemmatizer
lem_obj = WordNetLemmatizer()

In [14]:
good_review = []

for i in range(len(df)):
  text = df['review'][i] # every iteration 1 mail will come inside
  text = ''.join([i for i in text if i not in string.punctuation]) # removing punctuation
  text = text.lower() # lowering text
  text = ' '.join([j for j in text.split() if j not in stopwords.words('english')]) # removing stopwords
  text = ' '.join([lem_obj.lemmatize(k) for k in text.split()]) # each word lemmatization
  good_review.append(text)

In [15]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [17]:
voc_size = 500
proper_data = [one_hot(i,voc_size) for i in good_review]

In [18]:
c = []
for i in proper_data:
  c.append(len(i))
print(max(c))

1449


In [19]:
sent_length = 1449

final_data_to_model = pad_sequences(proper_data,maxlen = sent_length,padding='post')

In [20]:
final_data_to_model

array([[437, 340, 377, ...,   0,   0,   0],
       [360, 378, 117, ...,   0,   0,   0],
       [334, 360, 413, ...,   0,   0,   0],
       ...,
       [385, 276, 372, ...,   0,   0,   0],
       [220, 373, 209, ...,   0,   0,   0],
       [437, 331, 231, ...,   0,   0,   0]], dtype=int32)

In [25]:
df['sentiment'] = df['sentiment'].map({'positive':1,'negative':0}).astype(int)

In [26]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(final_data_to_model,df['sentiment'],test_size=0.2,random_state=42)

In [27]:
from sklearn.naive_bayes import MultinomialNB

In [28]:
reg = MultinomialNB()

In [29]:
reg.fit(X_train,y_train)

MultinomialNB()

In [30]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [31]:
y_train_pred = reg.predict(X_train)

In [32]:
confusion_matrix(y_train,y_train_pred)

array([[17678,  2361],
       [17249,  2712]])

In [33]:
print(accuracy_score(y_train,y_train_pred))

0.50975


In [34]:
print(classification_report(y_train,y_train_pred))

              precision    recall  f1-score   support

           0       0.51      0.88      0.64     20039
           1       0.53      0.14      0.22     19961

    accuracy                           0.51     40000
   macro avg       0.52      0.51      0.43     40000
weighted avg       0.52      0.51      0.43     40000



test perfomance

In [35]:
print(reg.score(X_test,y_test))

0.5057


In [51]:
f = 'This movie was a real gem! The acting was superb, the plot was engaging, and the cinematography was breathtaking. I was completely captivated from beginning to end and would highly recommend it to anyone who enjoys a good story." '

text = f # every iteration 1 mail will come inside
text = ''.join([i for i in text if i not in string.punctuation]) # removing punctuation
text = text.lower() # lowering text
text = ' '.join([j for j in text.split() if j not in stopwords.words('english')]) # removing stopwords
text = ' '.join([lem_obj.lemmatize(k) for k in text.split()]) # each word lemmatization
good_review.append(text)
voc_size = 500
proper_data = [one_hot(i,voc_size) for i in good_review]
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_length = 1449  # Define the maximum sequence length
d = pad_sequences(proper_data, maxlen=max_length, padding='post')
if reg.predict(d)[0]== 0:  # Assuming 0-1 output where < 0.5 is negative
  print('negative')
else:
  print('positive')
print(text)



negative
movie real gem acting superb plot engaging cinematography breathtaking completely captivated beginning end would highly recommend anyone enjoys good story
